In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from src.config.settings import CONFIG_PATH, DATA_PATH, ROOT_PATH
from src.utils.utils import collate_fn, load_yaml
import torch
import numpy as np

config = load_yaml(path=CONFIG_PATH)

# Eval

In [3]:
path_models = list(Path(config.data.training.save_dir).glob('*.pt'))

accuracy , auc, cm = [], [], []

for path_model in path_models: 
    
    print(path_model)
    saved_model = torch.load(path_model)
    accuracy.append(saved_model['acc'])
    auc.append(saved_model['auc'])
    #cm.append(saved_model['confusion_matrix'])

/data/ubuntu/data/saved_models/experiment_24/model0.pt


In [4]:
import pandas as pd

df_metrics = pd.DataFrame({"accuracy":accuracy, "auc":auc}, index = [i.name for i in path_models]).T
df_metrics

,model0.pt
accuracy,0.821429
auc,0.859412


In [14]:
import pandas as pd

df_metrics = pd.DataFrame({"accuracy":accuracy, "auc":auc}, index = [i.name for i in path_models]).T
df_metrics

,model0.pt,model1.pt,model2.pt
accuracy,0.845238,0.857143,0.583333
auc,0.916221,0.876157,0.537026


In [4]:
print(f"AUC = {np.mean(auc)} +/- {np.std(auc)}")
print(f"ACC = {np.mean(accuracy)} +/- {np.std(accuracy)}")

AUC = 0.9022679193628671 +/- 0.03313614686220466
ACC = 0.8387163561076605 +/- 0.02105550363929502


In [24]:
import pandas as pd

csv_paths = list(Path("/data/ubuntu/data/saved_models/experiment_22/").glob("sub*.csv"))[:3]
print(csv_paths)

li = []

for filename in csv_paths:
    df = pd.read_csv(filename)
    li.append(df)

frame = pd.concat(li, axis=1, ignore_index=False)
frame = frame.T.drop_duplicates().T
frame

[PosixPath('/data/ubuntu/data/saved_models/experiment_22/submission_model0.csv'), PosixPath('/data/ubuntu/data/saved_models/experiment_22/submission_model1.csv'), PosixPath('/data/ubuntu/data/saved_models/experiment_22/submission_model2.csv')]


,ID,Target,Target,Target
0,103.0,0.455303,0.380270,0.435988
1,117.0,0.946628,0.956077,0.443078
2,62.0,0.959353,0.380482,0.449491
3,328.0,0.329670,0.206216,0.205450
4,314.0,0.999991,0.999455,0.998287
...,...,...,...,...
115,91.0,0.304125,0.229579,0.251129
116,330.0,0.892403,0.367224,0.651358
117,324.0,0.329450,0.382285,0.439602
118,318.0,0.958602,0.378342,0.723550


In [26]:
mean_df

,ID,Target
0,103.0,0.423854
1,117.0,0.781928
2,62.0,0.596442
3,328.0,0.247112
4,314.0,0.999245
...,...,...
115,91.0,0.261611
116,330.0,0.636995
117,324.0,0.383779
118,318.0,0.686831


In [27]:
mean_df = pd.DataFrame({"ID":frame.ID, "Target":frame.Target.mean(axis=1) })
mean_df.to_csv("/data/ubuntu/data/saved_models/experiment_22/mean_4_models.csv", index=False)

# Train

In [3]:
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

from src.config.settings import CONFIG_PATH, DATA_PATH
from src.data.dataset import bag_dataset
from src.models.evaluation import evaluation
from src.models.model import Paper_network
from src.models.training import train_loop
from src.utils.utils import collate_fn, load_yaml


config = load_yaml(path=CONFIG_PATH)

metadata_file = DATA_PATH / config.data.training.metadata_file
data_root = DATA_PATH / config.data.training.data_dir

save_path = Path(config.data.training.save_dir)
save_path.mkdir(exist_ok=True)

save_model_path = save_path / config.data.training.saved_model
checkpoint_path = config.data.training.previous_model

df_annotated_bags = pd.read_csv(metadata_file)

id_train_bag, id_val_bag, target_train_bag, target_val_bag = train_test_split(
    df_annotated_bags.ID.to_list(), df_annotated_bags.Target.to_list(), test_size=config.data.split, random_state=42
)

config_tensorboard = config.data.training.tensorboard
if config_tensorboard.use_tensorboard:
    writer = SummaryWriter(config_tensorboard.tensorboard_writer)

epochs = config.data.training.epochs
lr = config.data.training.lr
batch_size = config.data.training.batch_size

train_ds = bag_dataset(id_train_bag, target_train_bag, data_root, random_selection=True)
val_ds = bag_dataset(id_val_bag, target_val_bag, data_root, random_selection=False)

train_pooling_loader = DataLoader(
    train_ds, batch_size=32, shuffle=True, num_workers=4, collate_fn=collate_fn
)
validation_pooling_loader = DataLoader(
    val_ds, batch_size=batch_size, shuffle=False, num_workers=4, collate_fn=collate_fn
)


In [40]:
minimum = 1000
target_min = -1
for batch_features, batch_targets in train_ds: 
    if batch_features.shape[0] < minimum : 
        minimum = batch_features.shape[0]
        target_min = batch_targets
    

In [41]:
minimum, target_min

(40, 0)

In [22]:
batch_features[1].shape, type(batch_features)

((128, 2048), tuple)

In [6]:
cat.shape , 128**2

(torch.Size([16296, 2048]), 16384)

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = Paper_network(input_size=2048, dropout=0.5)
model = model.to(device)

criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

n = config.data.training.n_monte_carlo

In [6]:
from src.models.certainty_pooling import certainty_pooling

for epoch in tqdm(range(epochs)):
    #running_loss = train_loop(train_pooling_loader, model, criterion, optimizer, epoch, device, n=n)
    
    running_loss = 0.0
    for i, sample in enumerate(train_pooling_loader, 0):
        inputs, targets = sample

        with torch.no_grad():
            inputs_pooling = np.asarray(
                [certainty_pooling(model, input, n, device, epsilon=1.0e-3) for input in inputs]
            )
        break
    break

  0%|                                                                                                                               | 0/150 [00:00<?, ?it/s]

torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([44, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])
torch.Size([128, 2048])


  0%|                                                                                                                               | 0/150 [00:02<?, ?it/s]


In [15]:
inputs_pooling.shape

(32, 2048)

In [23]:
for batch_features, batch_targets in train_pooling_loader: 
    break

T = 10

input_ = torch.cat([torch.tensor(i) for i in batch_features])
h = model(input_.to(device).float())

model.train()
cat = torch.cat([torch.tensor(i) for i in batch_features])

mc_output = torch.cat([model(cat.to(device).float()) for _ in range(T)], dim=0)
mc_output = mc_output.reshape(T,cat.shape[0],1).squeeze(-1)

output_split = torch.split(mc_output, split_size_or_sections=[i.shape[0] for i in batch_features], dim=1)
std_output = torch.cat([1 / i.std(0) for i in output_split])

z_m_global = (std_output * h.squeeze(-1))
z_m = torch.split(z_m_global, split_size_or_sections=[i.shape[0] for i in batch_features], dim=0)
k_star = torch.tensor([z_.argmax() for z_ in z_m])

pooling_input = torch.cat([torch.tensor(bag[k_star[idx]]).unsqueeze(0) for idx, bag in enumerate(batch_features)], dim=0)
pooling_target = torch.tensor(batch_targets)

TypeError: expected Tensor as element 0 in argument 0, but got int

In [17]:
pooling_input.shape

torch.Size([32, 2048])

In [97]:
z_m_global.shape , mc_output.shape

(torch.Size([4094]), torch.Size([10, 4094]))

In [68]:
output_split[0].shape , type(output_split) , h.shape

(torch.Size([10, 128]), tuple, torch.Size([3995, 1]))

In [80]:
z_m.max() , z_m.argmax()

(tensor(180.6760, device='cuda:0', grad_fn=<MaxBackward1>),
 tensor(1539, device='cuda:0'))

tensor([  38,  947,  835,  771,  914,  737, 1255,  800, 1119,  484,  378,  942,
         625,  913,  583,  241,  119,  918,   60,  538,  529,  571, 1227,  143,
          25,  373,  886,  854,  345, 1201,  561,  890])

In [65]:
std_output * h , std_output @ h

(tensor([[50.8923, 49.1789, 48.9541,  ..., 52.1460, 56.1518, 44.6217],
         [50.5437, 48.8420, 48.6187,  ..., 51.7888, 55.7671, 44.3160],
         [46.9885, 45.4065, 45.1989,  ..., 48.1460, 51.8445, 41.1989],
         ...,
         [51.3366, 49.6082, 49.3815,  ..., 52.6013, 56.6420, 45.0113],
         [50.4756, 48.7762, 48.5533,  ..., 51.7191, 55.6921, 44.2564],
         [49.2888, 47.6294, 47.4117,  ..., 50.5030, 54.3826, 43.2158]],
        device='cuda:0', grad_fn=<MulBackward0>),
 tensor([217832.3125], device='cuda:0', grad_fn=<SqueezeBackward3>))

In [75]:
(std_output * h.squeeze(-1)).shape , std_output.shape , h.shape

(torch.Size([3995]), torch.Size([3995]), torch.Size([3995, 1]))

tensor([50.8923, 48.8420, 45.1989,  ..., 52.6013, 55.6921, 43.2158],
       device='cuda:0', grad_fn=<MulBackward0>)